In [1]:
project_folder = "/data_1/ATM/data_1/sfm/projects/EGU2"

parse the gcps xml and convert to pandas dataframe

In [2]:
import os
import pandas as pd
import xml.etree.ElementTree as ET

def parse_gcp_xml(xml_file, type):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Create a list to store the data
    data = []

    if type == "image":
        # Iterate through each 'MesureAppuiFlottant1Im' in the XML
        for measure in root.findall('MesureAppuiFlottant1Im'):
            image_id = measure.find('NameIm').text
            # Iterate through each 'OneMesureAF1I' within the current 'MesureAppuiFlottant1Im'
            for pt in measure.findall('OneMesureAF1I'):
                gcp = pt.find('NamePt').text
                xy = pt.find('PtIm').text.split()
                x = int(xy[0])
                y = int(xy[1])

                # Append the extracted information to the data list
                data.append({'image_id': image_id, 'gcp': gcp, 'x': x, 'y': y})

        # Create a DataFrame
        df = pd.DataFrame(data, columns=['image_id', 'gcp', 'x', 'y'])

    elif type == "world":
        # Iterate through each 'OneAppuisDAF' in the XML
        for point in root.findall('OneAppuisDAF'):
            gcp = point.find('NamePt').text
            xy = point.find('Pt').text.split()
            x = float(xy[0])
            y = float(xy[1])
            incertitude = point.find('Incertitude').text.split()
            x_quality = float(incertitude[0])
            y_quality = float(incertitude[1])

            # Append the extracted information to the data list
            data.append({'gcp': gcp, 'x': x, 'y': y, 'x_quality': x_quality, 'y_quality': y_quality})

        # Create a DataFrame
        df = pd.DataFrame(data, columns=['gcp', 'x', 'y', 'x_quality', 'y_quality'])

    else:
        raise ValueError("Invalid type argument. Use 'image' or 'world'.")

    return df

path_gcp_files_image = os.path.join(project_folder, "Measures-S2D.xml")
path_gcp_files_real = os.path.join(project_folder, "Measures.xml")

gcps_image = parse_gcp_xml(path_gcp_files_image, "image")
gcps_real = parse_gcp_xml(path_gcp_files_real, "world")

gcps_image.columns=['image_id', 'gcp', 'x_img', 'y_img']
gcps_real.columns=['gcp', 'x_real', 'y_real', 'x_quality', 'y_quality']

data = gcps_image.merge(gcps_real, on='gcp', how='inner')

print(data)


                       image_id    gcp  x_img  y_img     x_real    y_real  \
0   OIS-Reech_CA180132V0095.tif   GCP1    203     17 -1976459.0  767651.5   
1   OIS-Reech_CA180132V0097.tif   GCP1     66     32 -1976459.0  767651.5   
2   OIS-Reech_CA180132V0095.tif   GCP2     85     39 -1980396.0  768833.0   
3   OIS-Reech_CA180132V0094.tif   GCP2    157     38 -1980396.0  768833.0   
4   OIS-Reech_CA180132V0095.tif   GCP3     74     82 -1981393.0  767669.0   
5   OIS-Reech_CA180132V0096.tif   GCP3      7     90 -1981393.0  767669.0   
6   OIS-Reech_CA180132V0095.tif   GCP4     21    104 -1983357.0  767838.5   
7   OIS-Reech_CA180132V0094.tif   GCP4     94    102 -1983357.0  767838.5   
8   OIS-Reech_CA180132V0095.tif   GCP5     23    110 -1983371.5  767614.5   
9   OIS-Reech_CA180132V0094.tif   GCP5     97    108 -1983371.5  767614.5   
10  OIS-Reech_CA180132V0095.tif   GCP6    179    122 -1978798.5  764802.0   
11  OIS-Reech_CA180132V0097.tif   GCP6     44    142 -1978798.5  764802.0   

In [ ]:
import random
import numpy as np

import src.load.load_image as li

import src.display.display_images as di

# get the unique image ids and shuffle
image_ids = data['image_id'].unique()
random.shuffle(image_ids)

for base_image_id in image_ids:

    # load image
    base_image = li.load_image(os.path.join(project_folder, base_image_id))

    # get all rows for the base image
    base_data = data[data['image_id'] == base_image_id]

    # get the ids for base image
    base_cgps = base_data['gcp'].unique()

    # get coordinates of the base gcps
    base_coords = base_data[['gcp', 'x_img', 'y_img']].values

    # get other ids and shuffle
    other_ids = image_ids[image_ids != base_image_id]
    random.shuffle(other_ids)

    for other_image_id in other_ids:

        # get all rows for the other image
        other_data = data[data['image_id'] == other_image_id]

                # get coordinates of the other gcps
        other_coords = other_data[['gcp', 'x_img', 'y_img']].values

        # check if the other image has the same gcps as the base image
        other_cgps = other_data['gcp'].unique()
        common_gcps = list(set(base_cgps) & set(other_cgps))
        if len(common_gcps) == 0:
            continue

        # get the common coords
        base_common_coords = base_data[base_data['gcp'].isin(common_gcps)][['gcp', 'x_img', 'y_img']].values
        other_common_coords = other_data[other_data['gcp'].isin(common_gcps)][['gcp', 'x_img', 'y_img']].values

        # sort the coords by gcps
        base_common_coords = base_common_coords[base_common_coords[:, 0].argsort()]
        other_common_coords = other_common_coords[other_common_coords[:, 0].argsort()]

        # get coordinates as numpy arrays
        base_common_coords = base_common_coords[:, 1:]
        other_common_coords = other_common_coords[:, 1:]

        # concat the base and other coords
        tie_points = np.concatenate((base_common_coords, other_common_coords), axis=1)

        print(tie_points)


        # load image
        other_image = li.load_image(os.path.join(project_folder, other_image_id))

        style_config = {
            "title": f"{base_image_id} : {other_image_id}",
        }
        di.display_images([base_image, other_image], tie_points=tie_points, style_config=style_config)



[[85 39 157 38]
 [21 104 94 102]
 [23 110 97 108]]
[[74 82 7 90]]
[[203 17 66 32]
 [179 122 44 142]]
[[66 32 203 17]
 [44 142 179 122]]
[[165 103 28 115]
 [162 35 29 48]
 [177 53 43 64]]
[[114 146 180 133]
 [126 78 194 68]]
[[28 115 165 103]
 [29 48 162 35]
 [43 64 177 53]]
[[104 93 36 103]
 [194 116 119 127]]
[[19 63 159 54]]
[[180 133 114 146]
 [194 68 126 78]]
[[7 90 74 82]]
[[70 144 146 145]
 [80 165 155 164]]
[[146 145 70 144]
 [155 164 80 165]]
